In [7]:
import pandas as pd

In [8]:
df1 = pd.read_csv('5_cleaned_website_data.csv')
df1['document_type'] = "web page"

In [9]:
df1.head()

,url,depth,title,topic,word_count,char_count,page_text,question,answer,TFIDF_Cosine_Similarity,...,SBERT_Similarity,Combined_Similarity_Score,ragas_bleu,ragas_precision_reference,ragas_faith,ragas_fact,regas_sem_sim,regas_rogue,north_star,document_type
0,https://degrees.apps.asu.edu/masters-phd/major...,1.0,Interdisciplinary Studies (Interdisciplinary A...,Academic Programs & Courses,155.0,1158.0,Program Description: The MA degree in interdis...,What is the primary focus of the MA degree in...,The primary focus of the MA degree in interdis...,0.806,...,0.944,0.509,0.381667,0.0,0.0,1.0,0.962515,0.410463,1.944,web page
1,https://degrees.apps.asu.edu/bachelors/major/A...,1.0,"Business (Tourism), BA",Academic Programs & Courses,110.0,771.0,Program Description: The BA in business with a...,What type of degree is offered in the program...,The program described in the context offers a ...,0.774,...,0.939,0.500,0.325056,1.0,1.0,1.0,0.980363,0.389610,1.939,web page
2,https://degrees.apps.asu.edu/bachelors/major/A...,1.0,"Applied Science (Animal Biology), BAS",Academic Programs & Courses,119.0,798.0,Program Description: The BAS program with a co...,What type of degree does the BAS program with...,The BAS program with a concentration in animal...,0.751,...,0.938,0.504,0.293256,1.0,1.0,1.0,0.978900,0.413625,1.938,web page
3,https://degrees.apps.asu.edu/masters-phd/major...,1.0,Interprofessional Healthy Aging (Graduate Cert...,Academic Programs & Courses,135.0,1044.0,Program Description: This certificate program ...,Which organization’s competencies does the cu...,The curriculum of the certificate program in h...,0.863,...,0.924,0.522,0.170465,0.0,0.0,1.0,0.967808,0.413994,1.924,web page
4,https://degrees.apps.asu.edu/bachelors/major/A...,1.0,"Secondary Education (Mathematics), BAE",Academic Programs & Courses,243.0,1760.0,Program Description: The BAE in secondary educ...,What grades does the BAE in secondary educati...,The Bachelor of Arts in Education (BAE) in Sec...,0.837,...,0.910,0.532,0.141226,0.0,0.0,1.0,0.954549,0.475591,1.910,web page


In [10]:
df2 = pd.read_csv('half_golden.csv')
df2.rename(columns={"context": "page_text", 
                    "Cosine": "TFIDF_Cosine_Similarity",
                    "similarity_score" : "Fuzzy_Similarity",
                    "BLEU" :"BLEU_Score",
                    "section" : "topic"}, inplace = True)

def word_count(series):
    return series.str.split().str.len()

df2['word_count'] = word_count(df2['page_text'])
df2['Fuzzy_Similarity'] = df2['Fuzzy_Similarity']/100
df2.shape

(111, 13)

In [11]:
df2.head()

,topic,title,file_name,document_type,page,total_pages,page_text,question,answer,Fuzzy_Similarity,BLEU_Score,TFIDF_Cosine_Similarity,word_count
0,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,8,12,['Fig. 4. A Euclidian single cluster dendrogra...,What are the limitations of the correlation m...,The text states that while a correlation matr...,0.58,0.027386,0.664086,309
1,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,8,12,['Fig. 4. A Euclidian single cluster dendrogra...,"Based on the hierarchical cluster analysis, h...",The text states that the number of clusters w...,0.56,0.008859,0.643154,309
2,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,7,12,['The word stems have meaning based on their a...,What specific methods were used to analyze th...,The paper mentions several methods used to an...,0.58,0.007092,0.619066,536
3,Results,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,6,12,['Fig. 2. Bar chart of the top 25 occurring wo...,How does the bi-plot in Fig. 3 provide insigh...,The bi-plot visually represents the relations...,0.50,0.007867,0.618405,133
4,Abstract,Text-Mining-for-Procedure-Level-Primitives-in-...,Text-Mining-for-Procedure-Level-Primitives-in-...,academic paper,1,12,['Text Mining for Procedure Level Primitives i...,How does this text mining approach relate to ...,The text mining approach is used to identify ...,0.61,0.011155,0.643712,321


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

df1_csv_path = '5_cleaned_website_data.csv'
df2_csv_path = 'half_golden.csv'
train_csv_path = 'train.csv'
test_csv_path = 'test.csv'

test_size=0.2
random_state=42
try:
    # Concatenate the DataFrames vertically (stacking them on top of each other)
    combined_df = pd.concat([df1, df2], ignore_index=True)

    # Perform the train-test split
    train_df, test_df = train_test_split(combined_df, test_size=test_size, random_state=random_state)

    # Save the train and test DataFrames to CSV files
    train_df.to_csv(train_csv_path, index=False)
    test_df.to_csv(test_csv_path, index=False)

    print(f"Combined data split successfully. Train data saved to: {train_csv_path}")
    print(f"Test data saved to: {test_csv_path}")

except FileNotFoundError as e:
    print(f"Error: File not found - {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Combined data split successfully. Train data saved to: train.csv
Test data saved to: test.csv
